In [29]:
# === Chunk 1: Setup & Daten laden ===
# (Dieser Chunk lädt die Stage-B-Prognosen aller Modelle und merged sie)

import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
import statsmodels.api as sm
from arch.bootstrap import MCS
import warnings
warnings.filterwarnings('ignore')

# --- 1. Pfad-Setup (Standard-Snippet) ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur: break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# Annahme: 'src' ist im Pfad, wir können config/io importieren
try:
    from src.io_timesplits import load_target
    from src.config import STAGEB_DIR
except ImportError:
    print("Konnte 'src' nicht importieren, füge es zum Pfad hinzu...")
    SRC_DIR = PROJECT_ROOT / "src"
    if str(SRC_DIR) not in sys.path:
        sys.path.append(str(SRC_DIR))
    from io_timesplits import load_target
    from config import STAGEB_DIR

print(f"Project Root: {PROJECT_ROOT}")
print(f"Stage B Dir:  {STAGEB_DIR}")

# --- 2. Modelle definieren, die verglichen werden sollen ---
# WICHTIG: BEIDE MODELLE MÜSSEN HIER AUFGELISTET SEIN!
MODEL_RUNS = {
    # "Key (wird zu Spaltenname)": "Ordnername in outputs/stageB/"

    # MODELL 1:
    "ET_All_Features": "extra_trees",

    # MODELL 2:
    "ET_Dynamic_FI": "extra_trees_dynamic_fi",
}

# --- 3. Lade-Funktion (Die korrigierte Version von zuletzt) ---
def load_active_predictions(model_runs: dict) -> pd.DataFrame:
    """
    Lädt alle 'is_active=True' Prognosen und merged sie zu einem
    einzigen DataFrame mit y_true und den Prognosen.
    """
    all_model_dfs = []
    y_true_series = None
    for model_name_key, run_id in model_runs.items():
        pred_path = STAGEB_DIR / run_id / "monthly" / "preds.csv"
        if not pred_path.exists():
            print(f"WARNUNG: Datei nicht gefunden, überspringe: {pred_path}")
            continue
        df = pd.read_csv(pred_path, parse_dates=["date_t_plus_1"])
        df_active = df[df["is_active"] == True].copy()
        if df_active.empty:
            print(f"WARNUNG: Keine 'is_active=True' Zeilen gefunden in: {pred_path}")
            continue
        df_active = df_active.set_index("date_t_plus_1")
        df_pred = df_active[["y_pred"]].rename(columns={"y_pred": f"pred_{model_name_key}"})
        all_model_dfs.append(df_pred)
        if y_true_series is None:
            y_true_series = df_active["y_true"].rename("y_true")
    if not all_model_dfs:
        raise FileNotFoundError("Keine Prognosedateien gefunden oder keine 'is_active=True' Zeilen.")
    df_merged = pd.concat(all_model_dfs, axis=1)
    if y_true_series is not None:
        df_merged = pd.concat([y_true_series, df_merged], axis=1)
    df_final = df_merged.dropna()
    if "y_true" in df_final.columns:
        cols = ["y_true"] + [col for col in df_final.columns if col != "y_true"]
        df_final = df_final[cols]
    return df_final

# --- 4. Daten laden ---
try:
    df_preds = load_active_predictions(MODEL_RUNS)
    print(f"\nDaten erfolgreich geladen. Zeitraum: {df_preds.index.min()} bis {df_preds.index.max()}. Shape: {df_preds.shape}")
    print("Verfügbare Spalten (wichtig für nächste Chunks):", df_preds.columns.tolist())
    display(df_preds.head())
except Exception as e:
    print(f"\nFEHLER beim Laden: {e}")
    print("Stellen Sie sicher, dass 'MODEL_RUNS' (oben) korrekt ist und die 'preds.csv'-Dateien existieren.")

Project Root: /Users/jonasschernich/Documents/Masterarbeit/Code
Stage B Dir:  /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/stageB

Daten erfolgreich geladen. Zeitraum: 2011-03-01 00:00:00 bis 2024-12-01 00:00:00. Shape: (166, 3)
Verfügbare Spalten (wichtig für nächste Chunks): ['y_true', 'pred_ET_All_Features', 'pred_ET_Dynamic_FI']


,y_true,pred_ET_All_Features,pred_ET_Dynamic_FI
date_t_plus_1,,,
2011-03-01,0.307692,0.120888,0.162870
2011-04-01,0.102249,0.076408,0.160755
2011-05-01,0.919305,0.200083,0.192483
2011-06-01,-1.417004,-0.365803,0.124461
2011-07-01,2.977413,0.939482,0.275927


In [30]:
# === Chunk 2: Verlust-DataFrame berechnen ===
# (Basis für MCS-Test)

# Berechne quadratische Verluste (Squared Errors)
df_losses = pd.DataFrame(index=df_preds.index)
y_true_np = df_preds["y_true"].to_numpy()

for col in df_preds.columns:
    if col.startswith("pred_"):
        # Spaltenname ist z.B. "pred_ET_All" -> Key wird "ET_All"
        model_name_key = col.replace("pred_", "")
        y_pred_np = df_preds[col].to_numpy()
        df_losses[model_name_key] = (y_true_np - y_pred_np) ** 2

print("Verlust-DataFrame (Squared Errors):")
display(df_losses.head())

# Gesamt-RMSE (zur Kontrolle)
print("\nGesamt-RMSE (Stage B):")
display(df_losses.mean().pow(0.5).sort_values())

Verlust-DataFrame (Squared Errors):


,ET_All_Features,ET_Dynamic_FI
date_t_plus_1,,
2011-03-01,0.034896,0.020974
2011-04-01,0.000668,0.003423
2011-05-01,0.517281,0.528270
2011-06-01,1.105023,2.376115
2011-07-01,4.153161,7.298025



Gesamt-RMSE (Stage B):


ET_All_Features    1.796497
ET_Dynamic_FI      2.259137
dtype: float64

In [27]:
# === DEBUG-CHUNK (NEUE ZELLE) ===
# Führen Sie dies aus, um die echten Spaltennamen zu sehen.

print("VERFÜGBARE SPALTENNAMEN IN df_preds:")
print(df_preds.columns.tolist())

VERFÜGBARE SPALTENNAMEN IN df_preds:
['y_true', 'pred_ET_Dynamic_FI']


In [31]:
# === Chunk 3: Diebold-Mariano (DM) Test ===

print("DM-Test (H0: Gleiche Prognosegüte)")
print("Berechnet als HAC-robuster t-Test auf die Verlustdifferenz (konsistent mit Text \S subsec:testing)")

# 1. Fehler (Errors) berechnen (von df_preds)
# Diese Namen müssen exakt mit den Spaltennamen aus Chunk 1 übereinstimmen!
e_model_1 = df_preds["y_true"] - df_preds["pred_ET_All_Features"]
e_model_2 = df_preds["y_true"] - df_preds["pred_ET_Dynamic_FI"]

# 2. Verluste (Squared Errors) berechnen
loss_model_1 = e_model_1**2
loss_model_2 = e_model_2**2

# 3. Verlustdifferenz (d_t) berechnen
# d_t = L(A) - L(B)
d = loss_model_1 - loss_model_2

# 4. DM-Test als Regression von d_t auf eine Konstante
X = np.ones((len(d), 1))
HAC_LAGS = 3

model_dm = sm.OLS(d, X).fit(cov_type='HAC', cov_kwds={'maxlags': HAC_LAGS})

t_stat = model_dm.tvalues[0]
p_value = model_dm.pvalues[0] # zweiseitiger p-Wert

print(f"\nVergleich: 'ET_All_Features' (Modell 1) vs. 'ET_Dynamic_FI' (Modell 2)")
print(f"  t-Statistik: {t_stat:.4f}  (Positiv = Modell 2 ist besser)")
print(f"  p-Wert (2-seitig): {p_value:.4f}")

if p_value < 0.05:
    print("  -> H0 verworfen. Die Modelle sind signifikant unterschiedlich.")
else:
    print("  -> H0 nicht verworfen. Kein signifikanter Unterschied.")

DM-Test (H0: Gleiche Prognosegüte)
Berechnet als HAC-robuster t-Test auf die Verlustdifferenz (konsistent mit Text \S subsec:testing)

Vergleich: 'ET_All_Features' (Modell 1) vs. 'ET_Dynamic_FI' (Modell 2)
  t-Statistik: -1.9203  (Positiv = Modell 2 ist besser)
  p-Wert (2-seitig): 0.0548
  -> H0 nicht verworfen. Kein signifikanter Unterschied.


In [32]:
# === Chunk 4: Clark-West (CW) Test (KORRIGIERT) ===

def clark_west_test(y_true, pred_A, pred_B, h=1):
    """
    Führt den Clark-West-Test durch (KORRIGIERTE IMPLEMENTIERUNG).
    Modell A (klein, genestet) vs. Modell B (groß)
    H0: E[f] <= 0 (Modell B ist nicht besser als A)
    H1: E[f] > 0 (Modell B ist signifikant besser als A)
    """
    e_A = y_true - pred_A
    e_B = y_true - pred_B

    # f-Term ist korrekt (wie in \S subsec:testing)
    f = (e_A**2) - ( (e_B**2) - (pred_B - pred_A)**2 )

    # KORREKTUR 1: Designmatrix X ist NUR eine Konstante (shape [N, 1])
    X = np.ones((len(f), 1))

    # maxlags=h-1 ist 0 für h=1 (korrekt)
    model = sm.OLS(f, X).fit(cov_type='HAC', cov_kwds={'maxlags': h-1})

    t_stat = model.tvalues[0]
    p_two = model.pvalues[0] # Zweiseitiger p-Wert von statsmodels

    # KORREKTUR 2: Korrekte einseitige p-Wert-Berechnung
    if t_stat > 0:
        # t-Statistik ist in Richtung der Alternative H1
        p_one = p_two / 2
    else:
        # t-Statistik ist in Richtung der Null H0
        p_one = 1 - (p_two / 2)

    print(f"Clark-West Test (H1: Modell B ist besser)")
    print(f"  Modell A (genestet): {pred_A.name.replace('pred_','')}")
    print(f"  Modell B (umfassend): {pred_B.name.replace('pred_','')}")
    print(f"  t-Statistik: {t_stat:.4f}")
    print(f"  p-Wert (1-seitig): {p_one:.4f}")

    if p_one < 0.05:
        print("  -> H0 verworfen. Modell B ist signifikant besser.")
    else:
        print("  -> H0 nicht verworfen. Modell B ist nicht signifikant besser.")
    return model

# Beispiel-Aufruf (unverändert)
if "RW" in df_losses.columns and "AR1" in df_losses.columns:
    clark_west_test(df_preds["y_true"],
                    df_preds["pred_RW"],
                    df_preds["pred_AR1"])
else:
    print("Bitte 'RW' und 'AR1' in MODEL_RUNS definieren, um CW-Test auszuführen.")

Bitte 'RW' und 'AR1' in MODEL_RUNS definieren, um CW-Test auszuführen.


In [33]:
# === Chunk 5: Model Confidence Set (MCS) (KORRIGIERT) ===

# KORREKTUR: Explizite Blocklänge (wie im Text \S subsec:testing spezifiziert)
# 6 oder 12 sind gängig für Monatsdaten.
BLOCK_LEN = 6

# df_losses ist der korrekte Input (T x M Matrix der Verluste)
mcs_procedure = MCS(df_losses,
                    size=0.10,  # alpha = 10%
                    block_size=BLOCK_LEN,
                    method='stationary' # Stellt 'stationary bootstrap' sicher
                   )
mcs_procedure.compute()

print(f"Model Confidence Set (MCS) - alpha = 10%, block_size={BLOCK_LEN}\n")
print("Enthaltene Modelle (p-Wert >= 0.10):")
print(mcs_procedure.included)

print("\nAusgeschlossene Modelle (p-Wert < 0.10):")
print(mcs_procedure.excluded)

Model Confidence Set (MCS) - alpha = 10%, block_size=6

Enthaltene Modelle (p-Wert >= 0.10):
['ET_All_Features']

Ausgeschlossene Modelle (p-Wert < 0.10):
['ET_Dynamic_FI']
